# 模型融合（特征融合）

In [2]:
import os
import shutil
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.applications import *
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from keras.preprocessing import image
import h5py
from sklearn.utils import shuffle

Using TensorFlow backend.


In [3]:
#参数
train_file_name='train'
valid_file_name='valid'

In [4]:
#图片分类，创建软链接
train_filenames = os.listdir('imagedata/train')
test_filenames = os.listdir('imagedata/test')

train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)


def cre_rem_dir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cre_rem_dir(train_file_name)
os.mkdir(train_file_name+'/cat')
os.mkdir(train_file_name+'/dog')

cre_rem_dir('test2')
os.mkdir('test2/test')

cur_path=os.getcwd()

    
for filename in train_cat:
    os.symlink(cur_path+'/imagedata/train/'+filename, train_file_name+'/cat/'+filename)
for filename in train_dog:
    os.symlink(cur_path+'/imagedata/train/'+filename, train_file_name+'/dog/'+filename)

for filename in test_filenames:
    os.symlink(cur_path+'/imagedata/test/'+filename, 'test2/test/'+filename)

In [5]:
#数据增强
def data_enhancement(model_pre_name,width,height):

    train_datagen =  ImageDataGenerator(
        preprocessing_function=model_pre_name.preprocess_input,
    )

    test_datagen = ImageDataGenerator(
        preprocessing_function=model_pre_name.preprocess_input,
    )


    train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(width, height),
        batch_size=batch_size,
        shuffle=False
    )
    
    test_generator = test_datagen.flow_from_directory(
        'test2',
        target_size=(width, height),
        batch_size=batch_size,
        shuffle=False
    )
    
    return (train_generator,test_generator)

In [6]:
def get_feature(model_name,model_pre_name,width,height,save_name):
    
    base_model = model_name(weights='imagenet', include_top=False)
    
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    train_generator,test_generator = data_enhancement(model_pre_name,width,height)
    
    train = model.predict_generator(train_generator)
    test = model.predict_generator(test_generator)
    
    with h5py.File(save_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
        

In [ ]:
#调试参数
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size 
batch_size=16

In [ ]:
#选择模型
get_feature(ResNet50,resnet50,224, 224,'gap_ResNet50.h5')
get_feature(InceptionV3,inception_v3,IM_WIDTH, IM_HEIGHT,'gap_InceptionV3.h5')
get_feature(InceptionResNetV2,inception_resnet_v2,IM_WIDTH, IM_HEIGHT,'gap_InceptionResNetV2.h5')

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [ ]:
X_train = []
X_test = []
np.random.seed(2018)

for filename in ["gap_ResNet50.h5", "gap_InceptionV3.h5", "gap_InceptionResNetV2.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [ ]:
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

In [ ]:
model.save('model.h5')

In [ ]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

In [ ]:
import pandas as pd
from keras.preprocessing.image import *

current_path=os.getcwd()
df = pd.read_csv(current_path+'/imagedata/sample_submission.csv')

image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)